In [189]:
import pandas as pd
import chardet
import pycountry
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim


In [190]:
current_date = datetime.now().strftime("%Y%m%d")
current_date

'20241122'

## Investigating countries not recognized by Tableau 
Put the source data in tableau and made two csvs of countries it couldn't recognize/map. Then make a list of unique countries between recipients and suppliers. This will give us a full list of the countries in our data that Tableau doesn't recognize so we can decide how to handle them. 

In [191]:
with open("int_data/unidentified_suppliers.csv", "rb") as f:
    supp_encoding = chardet.detect(f.read())
    print(supp_encoding["encoding"])

UTF-16


In [192]:
unid_supp_df = pd.read_csv('int_data/unidentified_suppliers.csv', encoding = supp_encoding['encoding'], sep='\t').reset_index()
unid_rec_df = pd.read_csv('int_data/unidentified_recipients.csv', encoding = supp_encoding['encoding'],sep='\t').reset_index()

unid_supp_df.head()

,index,Supplier,Number ordered
0,0,Yugoslavia,2198.0
1,1,unknown supplier(s),5390.0
2,2,United Nations**,4.0
3,3,Soviet Union,845420.0
4,4,South Yemen,12.0


In [193]:
# all countries in our source data not recognized by tableau
unique_unidentified_countries = list(set(unid_supp_df['Supplier']).union(set(unid_rec_df['Recipient'])))
unique_unidentified_countries

[nan,
 'unknown recipient(s)',
 '3',
 'ELF (Ethiopia)*',
 'unknown supplier(s)',
 'NLA (Macedonia)*',
 'Anti-Castro rebels (Cuba)*',
 'Contras (Nicaragua)*',
 'Amal (Lebanon)*',
 '0.25',
 'Soviet Union',
 'Indonesia rebels*',
 'Viet Minh (France)*',
 'SPLA (Sudan)*',
 'Armas (Guatemala)*',
 'Katanga',
 'Darfur rebels (Sudan)*',
 'RPF (Rwanda)*',
 'Hezbollah (Lebanon)*',
 'Libya GNC',
 'Hamas (Palestine)*',
 'Yugoslavia',
 'FMLN (El Salvador)*',
 'Viet Cong (South Vietnam)*',
 'UIC (Somalia)*',
 'FRELIMO (Portugal)*',
 'Libya HoR*',
 'PKK (Turkiye)*',
 'South Vietnam',
 'FNLA (Angola)*',
 'Pathet Lao (Laos)*',
 'Houthi rebels (Yemen)*',
 'PAIGC (Portugal)*',
 'NTC (Libya)*',
 'African Union**',
 'Yemen Arab Republic (North Yemen)',
 'Ukraine Rebels*',
 'European Union**',
 'Czechoslovakia',
 'MPLA (Portugal)*',
 'PLO (Israel)*',
 'South Yemen',
 'Syria rebels*',
 'MTA (Myanmar)*',
 'GUNT (Chad)*',
 'Mujahedin (Afghanistan)*',
 'ANC (South Africa)*',
 'United Wa State (Myanmar)*',
 'LRA 

### Read in our source data about arms trade
We will read in our data, and then compare the countries in the arms trade data to the countries recognized by Tableau.

In [194]:
with open("raw_data/trade-register.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result["encoding"])

ISO-8859-1


In [195]:
df = pd.read_csv('raw_data/trade-register.csv', encoding=result["encoding"])


In [196]:
unique_values_source_data = pd.unique(df[['Supplier', 'Recipient']].values.ravel())

# Convert to list if needed
unique_values_source_data = unique_values_source_data.tolist()
unique_values_source_data

['Brazil',
 'Afghanistan',
 'Russia',
 'Soviet Union',
 'United Kingdom',
 'Czechoslovakia',
 'United States',
 'Norway',
 'India',
 'Canada',
 'unknown supplier(s)',
 'Slovakia',
 'Ukraine',
 'Czechia',
 'Switzerland',
 'Belarus',
 'Bosnia-Herzegovina',
 'China',
 'Italy',
 'Turkiye',
 'South Africa',
 'African Union**',
 'Egypt',
 'Israel',
 'Albania',
 'East Germany (GDR)',
 'Germany',
 'France',
 'Algeria',
 'Finland',
 'UAE',
 'Spain',
 'Romania',
 'Netherlands',
 'Morocco',
 'Sweden',
 'Denmark',
 'Poland',
 'Syria',
 'Amal (Lebanon)*',
 'ANC (South Africa)*',
 'Angola',
 'Hungary',
 'Moldova',
 'Peru',
 'Bulgaria',
 'Kazakhstan',
 'Portugal',
 'Lithuania',
 'Cuba',
 'Anti-Castro rebels (Cuba)*',
 'Antigua and Barbuda',
 'Dominican Republic',
 'Argentina',
 'Austria',
 'Ireland',
 'Belgium',
 'Armas (Guatemala)*',
 'Armenia',
 'Montenegro',
 'Jordan',
 'Australia',
 'South Korea',
 'New Zealand',
 'Azerbaijan',
 'Georgia',
 'Pakistan',
 'Bahamas',
 'Bahrain',
 'Oman',
 'Malta',
 

In [197]:
tableau_identified_countries = [item for item in unique_values_source_data if item not in unique_unidentified_countries]
tableau_identified_countries

['Brazil',
 'Afghanistan',
 'Russia',
 'United Kingdom',
 'United States',
 'Norway',
 'India',
 'Canada',
 'Slovakia',
 'Ukraine',
 'Czechia',
 'Switzerland',
 'Belarus',
 'Bosnia-Herzegovina',
 'China',
 'Italy',
 'Turkiye',
 'South Africa',
 'Egypt',
 'Israel',
 'Albania',
 'Germany',
 'France',
 'Algeria',
 'Finland',
 'UAE',
 'Spain',
 'Romania',
 'Netherlands',
 'Morocco',
 'Sweden',
 'Denmark',
 'Poland',
 'Syria',
 'Angola',
 'Hungary',
 'Moldova',
 'Peru',
 'Bulgaria',
 'Kazakhstan',
 'Portugal',
 'Lithuania',
 'Cuba',
 'Antigua and Barbuda',
 'Dominican Republic',
 'Argentina',
 'Austria',
 'Ireland',
 'Belgium',
 'Armenia',
 'Montenegro',
 'Jordan',
 'Australia',
 'South Korea',
 'New Zealand',
 'Azerbaijan',
 'Georgia',
 'Pakistan',
 'Bahamas',
 'Bahrain',
 'Oman',
 'Malta',
 'Bangladesh',
 'Singapore',
 'Serbia',
 'Malaysia',
 'Barbados',
 'Belize',
 'Taiwan',
 'Benin',
 'Gabon',
 "Cote d'Ivoire",
 'Indonesia',
 'Bhutan',
 'Thailand',
 'Bolivia',
 'Uruguay',
 'Venezuela',


In [198]:
# countries that don't have tableau matches
no_tableau_matches = df[df[['Supplier','Recipient']].isin(unique_unidentified_countries).any(axis=1)]
no_tableau_matches.sort_values('Year of order', ascending=False)

,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
27827,unknown recipient(s),United States,2023.0,NaN,6.0,NaN,MD-500E,light helicopter,0.0,NaN,NaN,New,Cayuse Warrior Plus armed version; recipient i...,0.70,4.2,0.0
827,Antigua and Barbuda,unknown supplier(s),2023.0,NaN,1.0,NaN,PA-31 Navajo,light transport aircraft,1.0,NaN,2023,Second hand,Probably second-hand; PA-31-350 Chieftain version,0.30,0.3,0.3
27878,unknown recipient(s),Israel,2023.0,NaN,NaN,NaN,Barak-MX,SAM system,0.0,NaN,NaN,New,$1.2 b deal; designation uncertain (reported a...,47.00,0.0,0.0
15350,Mali,unknown supplier(s),2023.0,?,1.0,NaN,Il-76M,heavy transport aircraft,1.0,NaN,2023,Second hand,Second-hand; Il-76TD version,34.00,34.0,34.0
9793,Honduras,unknown supplier(s),2023.0,NaN,2.0,?,Bell-412,helicopter,0.0,NaN,NaN,Second hand but modernized,Second-hand but modernized before delivery; de...,3.30,6.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7729,Ethiopia,Czechoslovakia,1948.0,NaN,20.0,NaN,AH-IV-Hb,light tank,20.0,NaN,1950,New,NaN,0.08,1.6,1.6
9923,Hungary,Soviet Union,1947.0,?,50.0,?,SU-100,tank destroyer,50.0,?,1950; 1951,New,NaN,1.00,50.0,50.0
23902,Switzerland,Czechoslovakia,1946.0,NaN,90.0,?,PzJ-38(t) Hetzer,tank destroyer,90.0,?,1950; 1951; 1952,New,ST-1 version; Swiss designation Panzerjäger G-13,0.50,45.0,45.0
27505,NaN,3,9.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Because there have been countries created/changed since roughly the early 2000s we won't be looking at data before that time range. Here we will filter our data to orders placed after 1995 that have either the supplier or recipient not recognized by Tableau. We will then compare this to the list of countries previously created that are not recognized by Tableau to get the final list of countries in our data set that are in our range of time and not recognized.  

In [199]:
# Filter rows where 'Year of order' > 1995
filtered_df = no_tableau_matches[no_tableau_matches['Year of order'] > 2000]

# Get distinct values from both 'Recipient' and 'Supplier' columns
distinct_values = pd.concat([filtered_df['Recipient'], filtered_df['Supplier']]).dropna().unique()

# Convert to a list
distinct_list = list(distinct_values)

print(distinct_list)

['Afghanistan', 'African Union**', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Azerbaijan', 'Benin', 'Bolivia', 'Botswana', 'Burkina Faso', 'Cameroon', 'Canada', 'Central African Republic', 'Chile', 'Colombia', 'Congo', "Cote d'Ivoire", 'Darfur rebels (Sudan)*', 'Denmark', 'Djibouti', 'DR Congo', 'Ecuador', 'Equatorial Guinea', 'Ethiopia', 'Ghana', 'Guinea', 'Hamas (Palestine)*', 'Hezbollah (Lebanon)*', 'Honduras', 'Houthi rebels (Yemen)*', 'India', 'Indonesia', 'Iran', 'Iraq', 'Jamaica', 'Jordan', 'Kurdistan Regional Government (Iraq)*', 'Kyrgyzstan', 'Laos', 'Latvia', 'Liberia', 'Libya GNC', 'Libya HoR*', 'LRA (Uganda)*', 'LTTE (Sri Lanka)*', 'Madagascar', 'Malaysia', 'Mali', 'Mozambique', 'Myanmar', 'NATO**', 'New Zealand', 'Niger', 'NLA (Macedonia)*', 'Northern Alliance (Afghanistan)*', 'NTC (Libya)*', 'OSCE**', 'Peru', 'Philippines', 'PIJ (Israel/Palestine)*', 'PKK (Turkiye)*', 'PRC (Israel/Palestine)*', 'Russia', 'Rwanda', 'Saudi Arabia', 'Senegal', 'South Korea', 'South Sudan

In [200]:
set(unique_unidentified_countries).intersection(distinct_list)

{'African Union**',
 'Darfur rebels (Sudan)*',
 'European Union**',
 'Hamas (Palestine)*',
 'Hezbollah (Lebanon)*',
 'Houthi rebels (Yemen)*',
 'Kurdistan Regional Government (Iraq)*',
 'LRA (Uganda)*',
 'LTTE (Sri Lanka)*',
 'Libya GNC',
 'Libya HoR*',
 'NATO**',
 'NLA (Macedonia)*',
 'NTC (Libya)*',
 'Northern Alliance (Afghanistan)*',
 'OSCE**',
 'PIJ (Israel/Palestine)*',
 'PKK (Turkiye)*',
 'PRC (Israel/Palestine)*',
 'Syria rebels*',
 'UIC (Somalia)*',
 'Ukraine Rebels*',
 'United Nations**',
 'United Wa State (Myanmar)*',
 'unknown rebel group*',
 'unknown recipient(s)',
 'unknown supplier(s)'}

## Categorize weapon descriptions
Make a high level category and a sub-category for each weapon description 

In [201]:
category_mapping = {
    'Aircraft': [
        'fighter aircraft', 'bomber aircraft', 'transport aircraft', 'reconnaissance aircraft', 
        'trainer aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'trainer/combat aircraft', 
        'AGS aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'airship', 'target tow aircraft', 'light aircraft', 'ground attack aircraft',
        'ASW aircraft', 'AEW aircraft', 'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'transport ac/helicopter', 'transport/trainer aircraft', 'airship', 
        'light transport aircraft', 'heavy transport aircraft', 'tanker/transport aircraft', 'trainer/light aircraft', 'light/trainer aircraft'
    ],
    'Helicopters': [
        'combat helicopter', 'transport helicopter', 'ASW helicopter', 'light helicopter', 
        'AEW helicopter', 'anti-ship helicopter', 'SIGINT helicopter', 'helicopter', 'ASW Helicopter'
    ],
    'Unmanned Vehicles': [
        'UAV', 'armed UAV', 'reconnaissance AV', 'maritime patrol UAV', 'light aircraft/UAV', 'loitering munition' 
    ],
    'Naval Vessels': [
        'frigate', 'destroyer', 'corvette', 'patrol craft', 'submarine', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'gunboat', 'landing ship', 'salvage ship', 
        'survey ship', 'cruiser', 'aircraft carrier', 'minelayer', 'replenishment tanker', 
        'corvette/minesweeper', 'OPV/transport ship', 'support/landing ship', 'landing/patrol craft',
        'transport ship', 'FAC (Fast Attack Craft)', 'minesweeper', 'landing craft', 'tug', 'nuclear submarine', 'OPV/tug',
        'frigate/landing ship', 'SSB (Ballistic Missile Submarine)', 'SIGINT ship', 'patrol craft/transport craft', 'transport craft',
        'support craft', 'cargo craft', 'midget submarine', 'icebreaker/OPV', 'FAC', 'SSB'
    ],
    'Ground Vehicles/Artillery': [
        'tank', 'light tank', 'armoured car', 'APC', 'IFV', 'AFSV', 'APV', 'AMV', 'ARV', 'AEV/ARV', 
        'armoured bridgelayer', 'training tank', 'tank turret', 'self-propelled gun chassis', 'tank chassis',
        'APC/APV', 'APC/IFV', 'APC/CP', 'IFV/AFSV', 'IFV/APC turret', 'APC turret', 'IFV turret', 'self-propelled AD system',
        'self-propelled gun', 'self-propelled MRL', 'towed gun', 'self-propelled AA gun', 'coastal defence system', 'AA gun (Anti-Aircraft Gun)',
        'towed MRL', 'ALV (Armored Logistics Vehicle)', 'naval gun', 'tank destroyer', 'mortar', 'self-propelled mortar', 'anti-aircraft gun',
        'SPG turret', 'ASW mortar', 'AEV (Armored Engineering Vehicle)', 'helicopter turret', 'anti-tank AV turret', 'self-propelled AA gun turret',
        'apc (Armored Personnel Carrier)', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun', 'ALV', 'AEV', 'apc'
    ],
    'Missiles/Rockets/Bombs': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile',
        'SSM', 'SSM launcher', 'SSM TEL', 'anti-ship missile', 'anti-radar missile', 'anti-tank missile', 
        'guided bomb', 'guided rocket', 'guided rocket/SSM', 'guided rocket/ASM', 'SSM/ASM', 'anti-ship missile/SSM',
        'anti-ship/land-attack missile', 'land-attack missile', 'anti-tank missile/ASM', 'guided glide bomb',
        'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'ASW torpedo', 'ASW MRL', 
        'anti-ship missile/ASM', 'BVRAAM', 'BVRAAM/SAM', 'SRAAM', 'ASM', 'ABM/SAM system', 'naval MRL', 'ASM (Anti-Ship Missile)',
        'guided shell', 'ASW missile', 'anti-ship/anti-radar missile', 'naval mine/torpedo', 'ABM system', 'ASW rocket launcher',
        'anti-ship missile/ASM/SAM', 'SLBM (Submarine-Launched Ballistic Missile)', 'ASM', 'SLBM', 'anti-radar missile/ASM'
    ],
    'Radar/Detection Systems': [
        'air search radar', 'AGS radar', 'sea search radar', 'multi-function radar', 'air search system', 
        'fire control radar', 'air/sea search radar', 'artillery locating radar', 'ground surv radar', 
        'ground/sea search radar', 'AEW radar', 'MP aircraft radar', 'aircraft EO system', 
        'aircraft EO/radar system', 'air/ground surv radar', 'SAM system radar', 'multi-role radar', 'radar',
        'combat aircraft radar', 'combat heli radar', 'SONAR', 'submarine sonar', 'ASW sonar', 'SONAR system',
        'aircraft recce system', 'height-finding radar', 'ASW sonar', 'SONAR', 'recce satellite', 'EO search/fire control', 'EO system',
        'AGS/MP aircraft radar', 'Naval EO system', 'AGS/SIGINT system', 'surveillance satellite'
    ],
    'Specialized Equipment': [
        'aircraft engine', 'turbofan', 'turbojet', 'gas turbine', 'vehicle engine', 'AIP engine', 
        'air refuel system', 'maritime patrol aircraft', 'SIGINT system', 
        'SIGINT aircraft', 'AALS', 'VEHICLE ENGINE', 'turbojet', 'nuclear reactor', 'ship engine', 'turboprop'
    ],
    'Other': [
        'SSB' , 'training equipment'
    ]
}

# Expanded sub-category mappings
sub_category_mapping = {
    # Aircraft Sub-categories
    'Aircraft': {
        'Fighter/Combat Aircraft': ['fighter aircraft', 'trainer/combat aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'ground attack aircraft', 'ASW aircraft'],
        'Bomber/Heavy Aircraft': ['bomber aircraft', 'heavy transport aircraft'],
        'Reconnaissance/Surveillance Aircraft': ['reconnaissance aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'AGS aircraft', 'light aircraft', 'AEW aircraft',
                                                'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'airship'],
        'Trainer Aircraft': ['trainer aircraft', 'trainer/light aircraft', 'light/trainer aircraft', 'target tow aircraft'],
        'Transport Aircraft': ['transport aircraft', 'light transport aircraft', 'tanker/transport aircraft', 'transport ac/helicopter', 'transport/trainer aircraft']
    },
    # Helicopter Sub-categories
    'Helicopters': {
        'Combat Helicopters': ['combat helicopter', 'ASW helicopter', 'anti-ship helicopter', 'SIGINT helicopter'],
        'Transport/Utility Helicopters': ['transport helicopter', 'light helicopter'],
        'Specialized Helicopters': ['AEW helicopter', 'ASW Helicopter', 'helicopter']
    },
    # Unmanned Vehicles Sub-categories
    'Unmanned Vehicles': {
        'UAV/Drone': ['UAV', 'armed UAV', 'light aircraft/UAV', 'loitering munition'],
        'Reconnaissance/Surveillance UAV': ['reconnaissance AV', 'maritime patrol UAV']
    },
    'Naval Vessels' :{
        'Combat Vessel': ['frigate', 'destroyer', 'corvette', 'submarine', 'gunboat', 'cruiser', 'aircraft carrier', 
                         'corvette/minesweeper', 'FAC (Fast Attack Craft)', 'minesweeper', 'nuclear submarine', 'SSB (Ballistic Missile Submarine)',
                         'midget submarine', 'FAC', 'SSB'],
        'Support Vessel' : [ 'patrol craft', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'landing ship', 'salvage ship', 
        'survey ship', 'minelayer', 'replenishment tanker', 'OPV/transport ship', 'support/landing ship', 
        'landing/patrol craft', 'transport ship', 'landing craft', 'tug', 'OPV/tug', 'frigate/landing ship', 'SIGINT ship', 'patrol craft/transport craft',
        'transport craft', 'support craft', 'cargo craft', 'icebreaker/OPV']
    },
    'Ground Vehicles/Artillery' : {
    'Main Battle Tanks': ['tank', 'light tank', 'training tank', 'tank chassis', 'tank destroyer'],
    'Armored Personnel Carriers (APC)': ['APC', 'APC/APV', 'APC/IFV', 'APC/CP', 'apc (Armored Personnel Carrier)', 'apc'],
    'Infantry Fighting Vehicles (IFV)': ['IFV', 'IFV/AFSV', 'IFV/APC turret', 'IFV turret'],
    'Armored Reconnaissance Vehicles (ARV)': ['ARV', 'AEV/ARV'],
    'Armored Fighting Support Vehicles (AFSV)': ['AFSV', 'AMV', 'ALV (Armored Logistics Vehicle)', 'ALV'],
    'Light Armored Vehicles (APV)': ['APV', 'armoured car'],
    'Self-Propelled Artillery': ['self-propelled gun chassis', 'self-propelled AD system', 'self-propelled gun', 'self-propelled MRL', 'self-propelled AA gun',
                                'AA gun (Anti-Aircraft Gun)', 'self-propelled mortar', 'SPG turret', 'self-propelled AA gun turret'],
    'Engineering Vehicles': ['armoured bridgelayer', 'AEV (Armored Engineering Vehicle)', 'AEV'],
    'Tank Components': ['tank turret'],
    'Artillery' : ['towed gun', 'mortar', 'coastal defence system', 'towed MRL', 'naval gun', 'anti-aircraft gun', 'ASW mortar', 'helicopter turret', 'anti-tank AV turret',
                  'APC turret', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun']
    },
    'Missiles/Rockets/Bombs' : {
    'Surface-to-Air Missiles (SAM)': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile', 'ABM/SAM system', 'ASM (Anti-Ship Missile)', 'ABM system'
    ],
    'Surface-to-Surface Missiles (SSM)': [
        'SSM', 'SSM launcher', 'SSM TEL', 'SSM/ASM', 'SSM/anti-ship missile', 'anti-ship missile/SSM', 'land-attack missile'
    ],
    'Air-to-Air Missiles (AAM)': [
        'BVRAAM', 'SRAAM', 'BVRAAM/SAM'
    ],
    'Anti-Ship Missiles (ASM)': [
        'anti-ship missile', 'anti-ship missile/ASM', 'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'anti-ship/anti-radar missile',
        'naval mine/torpedo', 'anti-ship missile/ASM/SAM', 'ASM'
    ],
    'Anti-Radar Missiles': [
        'anti-radar missile', 'anti-radar missile/ASM'
    ],
    'Anti-Tank Missiles': [
        'anti-tank missile', 'anti-tank missile/ASM'
    ],
    'Guided Bombs and Rockets': [
        'guided bomb', 'guided rocket', 'guided glide bomb', 'guided rocket/SSM', 'guided rocket/ASM', 'guided shell'
    ],
    'Land-Attack Missiles': [
        'anti-ship/land-attack missile', 'land-attack missile'
    ],
    'Anti-Submarine Warfare (ASW) Weapons': [
        'ASW torpedo', 'ASW MRL', 'anti-ship/ASW torpedo', 'ASW missile', 'ASW rocket launcher'
    ],
    'Naval Rocket Launchers': [
        'naval MRL'
    ],
    'Ballistic Missiles' : ['SLBM (Submarine-Launched Ballistic Missile)', 'SLBM']    
    
    },
    'Radar/Detection Systems' : {
    'Air Search Radar': [
        'air search radar', 'AEW radar', 'air search system', 'air/sea search radar', 'air/ground surv radar'
    ],
    'Sea Search Detection Systems': [
        'sea search radar', 'ground/sea search radar', 'ASW sonar', 'SONAR', 'submarine sonar', 'AGS/MP aircraft radar'
    ],
    'Ground Surveillance Radar': [
        'ground surv radar', 'ground/sea search radar', 'air/ground surv radar', 'AGS/SIGINT system'
    ],
    'Multi-function Radar': [
        'multi-function radar', 'multi-role radar'
    ],
    'Fire Control Radar': [
        'fire control radar', 'SAM system radar', 'AGS radar', 'EO system', 'EO search/fire control', 'Naval EO system'
    ],
    'Artillery/Weapon Locating Radar': [
        'artillery locating radar'
    ],
    'Aircraft-Mounted Systems': [
        'MP aircraft radar', 'aircraft EO system', 'aircraft EO/radar system', 'combat aircraft radar', 'combat heli radar',
        'aircraft recce system'
    ],
    'Other Detection Systems': ['height-finding radar', 'recce satellite', 'surveillance satellite'
                               ]
    },
    'Specialized Equipment' : {
    'Aircraft Engines': [
        'aircraft engine', 'turbofan', 'turbojet', 'turboprop'
    ],
    'Vehicle Engines': [
        'vehicle engine', 'VEHICLE ENGINE', 'gas turbine', 'AIP engine', 'nuclear reactor', 'ship engine'
    ],
    'Refueling and Logistics Systems': [
        'air refuel system', 'AALS'
    ],
    'Intelligence and Surveillance Systems': [
        'SIGINT system', 'SIGINT aircraft', 'maritime patrol aircraft'
    ]
}
    # Add more sub-categories as needed in similar structure...
}

# The mapping functions will work the same, now with the expanded mappings.

def map_category(description):
    for category, items in category_mapping.items():
        if description in items:
            return category
    return 'Unknown'

def map_sub_category(description):
    for category, sub_categories in sub_category_mapping.items():
        for sub_category, items in sub_categories.items():
            if description in items:
                return sub_category
    return 'Unknown'

# Applying the mapping functions
df['Category'] = df['Weapon description'].apply(map_category)
df['Sub-Category'] = df['Weapon description'].apply(map_sub_category)



In [202]:
# Creating a DataFrame with three columns: 'Description', 'Category', and 'Sub-category' for exporting to csv for sharing 
csv_mapping_data = {'Category': [], 'Sub-category': [], 'Description': []}

for category, subcategories in sub_category_mapping.items():
    for sub_category, descriptions in subcategories.items():
        for description in descriptions:
            csv_mapping_data['Category'].append(category)
            csv_mapping_data['Sub-category'].append(sub_category)
            csv_mapping_data['Description'].append(description)

# Generating the DataFrame
df_three_columns = pd.DataFrame(csv_mapping_data)
df_three_columns.head()

,Category,Sub-category,Description
0,Aircraft,Fighter/Combat Aircraft,fighter aircraft
1,Aircraft,Fighter/Combat Aircraft,trainer/combat aircraft
2,Aircraft,Fighter/Combat Aircraft,FGA aircraft
3,Aircraft,Fighter/Combat Aircraft,FGA/EW aircraft
4,Aircraft,Fighter/Combat Aircraft,ground attack aircraft


In [203]:
# Save this mapping to a csv
df_three_columns.to_csv('final_data/description_mapping.csv')

In [204]:
# drop nonsense columns
df.drop([' .1', ' .2', ' '], axis=1, inplace=True)
df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0,Aircraft,Fighter/Combat Aircraft
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0,Specialized Equipment,Aircraft Engines
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6,Specialized Equipment,Aircraft Engines
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM)
4,Afghanistan,United Kingdom,2009.0,2.0,Mi-17,transport helicopter,2.0,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8,Helicopters,Transport/Utility Helicopters


# Add WDI indicators
We will add world bank country indicators do our database related to military expenditures and also bring in the region and income group of each country.

We'll start by reading in the indicator data (WDICSV) and the country info (WDICountry).

From the WDICountry table we'll keep the table name, region, and income group for joining to our data later 

In [205]:
wdi_df = pd.read_csv('raw_data/WDICSV.csv') 
country_df = pd.read_csv('raw_data/WDICountry.csv')
country_df.head()

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,NaN,Latin America & Caribbean,High income,AW,...,NaN,Enhanced General Data Dissemination System (e-...,2020 (expected),NaN,NaN,Yes,NaN,NaN,2018.0,NaN
1,AFE,Africa Eastern and Southern,Africa Eastern and Southern,Africa Eastern and Southern,ZH,NaN,"26 countries, stretching from the Red Sea in t...",NaN,NaN,ZH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,The reporting period for national accounts dat...,South Asia,Low income,AF,...,Consolidated central government,Enhanced General Data Dissemination System (e-...,1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2016/17",NaN,NaN,NaN,2018.0,NaN
3,AFW,Africa Western and Central,Africa Western and Central,Africa Western and Central,ZI,NaN,"22 countries, stretching from the westernmost ...",NaN,NaN,ZI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,The World Bank systematically assesses the app...,Sub-Saharan Africa,Lower middle income,AO,...,Budgetary central government,Enhanced General Data Dissemination System (e-...,2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",NaN,NaN,NaN,2018.0,NaN


In [206]:
country_region_mapping = country_df[['Table Name', 'Region', 'Income Group', '2-alpha code']]
country_region_mapping = country_region_mapping.rename(columns={'2-alpha code': 'two_alpha_code'})

country_region_mapping

,Table Name,Region,Income Group,two_alpha_code
0,Aruba,Latin America & Caribbean,High income,AW
1,Africa Eastern and Southern,NaN,NaN,ZH
2,Afghanistan,South Asia,Low income,AF
3,Africa Western and Central,NaN,NaN,ZI
4,Angola,Sub-Saharan Africa,Lower middle income,AO
...,...,...,...,...
260,Kosovo,Europe & Central Asia,Upper middle income,XK
261,"Yemen, Rep.",Middle East & North Africa,Low income,YE
262,South Africa,Sub-Saharan Africa,Upper middle income,ZA
263,Zambia,Sub-Saharan Africa,Lower middle income,ZM


In [207]:
country_region_mapping[country_region_mapping['two_alpha_code']=='TR']

,Table Name,Region,Income Group,two_alpha_code
243,Türkiye,Europe & Central Asia,Upper middle income,TR


In [208]:
wdi_df = wdi_df.merge(country_df[['Table Name','Region','Income Group', 'Long Name']], how='left', left_on='Country Name', right_on = 'Table Name')
wdi_df = wdi_df.drop('Table Name', axis=1)

In [209]:
wdi_df_long = pd.melt(wdi_df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Region', 'Income Group'],
                      var_name = 'Year',
                      value_name = 'indicator_value')
wdi_df_long.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Region,Income Group,Year,indicator_value
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,1960,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,1960,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,1960,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,1960,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,1960,NaN


In [210]:
indicators = [
    "Military expenditure (% of GDP)",
    "Military expenditure (% of general government expenditure)",
    "Military expenditure (current LCU)",
    "Armed Forces Personnel (% of total labor force)",
    "Armed Forces personnel, total",
    "Arms exports (SIPRI trend indicator values)",
    "Arms imports (SIPRI trend indicator values)",
    "Military expenditure (Current USD)",
    "Armed forces personnel (% of total labor force)",
    "Armed forces personnel, total"

]

In [211]:
def pivot_and_save_indicators(df, indicators, csv_name=None):
    # Filter the DataFrame to include only the specified indicators
    filtered_df = df[df['Indicator Name'].isin(indicators)]
    
    # Pivot the DataFrame
    pivoted_df = filtered_df.pivot(
        index=["Country Name", "Country Code", "Year", "Region", "Income Group"],
        columns="Indicator Name",
        values="indicator_value"
    ).reset_index()
    
    if csv_name != None:
    # Save the pivoted DataFrame to a CSV file
        pivoted_df.to_csv(csv_name, index=False)
    
    # Return the pivoted DataFrame
    return pivoted_df

In [212]:
arms_indicators = pivot_and_save_indicators(wdi_df_long, indicators)

In [213]:
non_region_df = arms_indicators[arms_indicators['Region'].notna()]
non_region_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13845 entries, 0 to 17289
Data columns (total 12 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   Country Name                                                13845 non-null  object
 1   Country Code                                                13845 non-null  object
 2   Year                                                        13845 non-null  object
 3   Region                                                      13845 non-null  object
 4   Income Group                                                13780 non-null  object
 5   Armed forces personnel (% of total labor force)             5039 non-null   object
 6   Armed forces personnel, total                               5501 non-null   object
 7   Arms exports (SIPRI trend indicator values)                 2453 non-null   object
 8   Arms import

In [214]:
# Convert 'Column1' to numeric with errors coerced to NaN
non_region_df['Year'] = pd.to_numeric(non_region_df['Year'], errors='coerce')

# Drop rows where 'Column1' is NaN
non_region_df = non_region_df.dropna(subset=['Year'])

# Convert the column to integer type
non_region_df['Year'] = non_region_df['Year'].astype(int)

/var/folders/63/jzv4h4hj37j3wl3_r5y5828h0000gn/T/ipykernel_35940/3292896927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_region_df['Year'] = pd.to_numeric(non_region_df['Year'], errors='coerce')


In [215]:
non_region_df = non_region_df[non_region_df['Year']>=2000]

In [216]:
# Filter rows where 'Region' is not NaN
filtered_df = non_region_df[non_region_df['Region'].notna()]

# Get unique values from the 'Country Name' column
unique_countries_wdi = filtered_df['Country Name'].unique()

# Convert to a list (optional)
unique_countries_list = unique_countries_wdi.tolist()

print(unique_countries_list)

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Gre

In [217]:
tableau_countries_not_in_indicators_data = [item for item in tableau_identified_countries if item not in unique_countries_list]
tableau_countries_not_in_indicators_data

['Russia',
 'Slovakia',
 'Bosnia-Herzegovina',
 'Turkiye',
 'Egypt',
 'UAE',
 'Syria',
 'South Korea',
 'Bahamas',
 'Taiwan',
 "Cote d'Ivoire",
 'Venezuela',
 'Iran',
 'Brunei',
 'Kyrgyzstan',
 'Congo',
 'DR Congo',
 'North Korea',
 'eSwatini',
 'Gambia',
 'Laos',
 'Micronesia',
 'Northern Cyprus',
 'Palestine',
 'Saint Kitts and Nevis',
 'Saint Vincent',
 'Western Sahara',
 'Yemen']

In [218]:
mapping_wbi_to_sipri = {
    'Russian Federation': 'Russia',
 'Slovak Republic': 'Slovakia',
 'Bosnia and Herzegovina': 'Bosnia-Herzegovina',
 'Turkey': 'Turkiye',
 'Türkiye':'Turkiye',
 'Egypt, Arab Rep.': 'Egypt',
 'United Arab Emirates': 'UAE',
 'Syrian Arab Republic': 'Syria',
 'Korea, Rep.': 'South Korea',
 'Bahamas, The': 'Bahamas',
 'Macao SAR, China': 'Taiwan',
 'Côte d’Ivoire': "Cote d'Ivoire",
 'Venezuela, RB': 'Venezuela',
 'Iran, Islamic Rep.': 'Iran',
 'Brunei Darussalam': 'Brunei',
 'Kyrgyz Republic': 'Kyrgyzstan',
 'Congo, Rep.': 'Congo',
 'Congo, Dem. Rep.': 'DR Congo',
 "Korea, Dem. People's Rep.": 'North Korea',
 'Eswatini': 'eSwatini',
 'Gambia, The': 'Gambia',
 'Lao PDR': 'Laos',
 'Micronesia, Fed. Sts.': 'Micronesia',
 'Cyprus': 'Northern Cyprus',
 'West Bank and Gaza': 'Palestine',
 'St. Kitts and Nevis': 'Saint Kitts and Nevis',
 'St. Vincent and the Grenadines': 'Saint Vincent',
 'Yemen, Rep.': 'Yemen'}


In [219]:
non_region_df['sipri_country_mapping'] = non_region_df['Country Name'].apply(
    lambda x: x if x not in mapping_wbi_to_sipri else mapping_wbi_to_sipri[x]
)

In [220]:
country_region_mapping['sipri_country_mapping'] = country_region_mapping['Table Name'].apply(
    lambda x: x if x not in mapping_wbi_to_sipri else mapping_wbi_to_sipri[x]
)
country_region_mapping.drop(columns=['Table Name'], inplace=True)
country_region_mapping

,Region,Income Group,two_alpha_code,sipri_country_mapping
0,Latin America & Caribbean,High income,AW,Aruba
1,NaN,NaN,ZH,Africa Eastern and Southern
2,South Asia,Low income,AF,Afghanistan
3,NaN,NaN,ZI,Africa Western and Central
4,Sub-Saharan Africa,Lower middle income,AO,Angola
...,...,...,...,...
260,Europe & Central Asia,Upper middle income,XK,Kosovo
261,Middle East & North Africa,Low income,YE,Yemen
262,Sub-Saharan Africa,Upper middle income,ZA,South Africa
263,Sub-Saharan Africa,Lower middle income,ZM,Zambia


In [221]:
non_region_df.head()

Indicator Name,Country Name,Country Code,Year,Region,Income Group,Armed forces personnel (% of total labor force),"Armed forces personnel, total",Arms exports (SIPRI trend indicator values),Arms imports (SIPRI trend indicator values),Military expenditure (% of GDP),Military expenditure (% of general government expenditure),Military expenditure (current LCU),sipri_country_mapping
40,Afghanistan,AFG,2000,South Asia,Low income,8.713139,400000.0,NaN,NaN,NaN,NaN,NaN,Afghanistan
41,Afghanistan,AFG,2001,South Asia,Low income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan
42,Afghanistan,AFG,2002,South Asia,Low income,2.438597,120000.0,NaN,34000000.0,NaN,NaN,NaN,Afghanistan
43,Afghanistan,AFG,2003,South Asia,Low income,2.447185,130000.0,NaN,NaN,NaN,NaN,NaN,Afghanistan
44,Afghanistan,AFG,2004,South Asia,Low income,0.487494,27000.0,NaN,NaN,2.431254,16.134336,5986000000.0,Afghanistan


In [222]:
#non_region_df.to_csv('final_data/arms_indicator_data.csv')

In [223]:
df.head()

,Recipient,Supplier,Year of order,Number ordered,Weapon designation,Weapon description,Number delivered,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons,Category,Sub-Category
0,Afghanistan,Brazil,2017.0,6.0,EMB-314 Super Tucano,trainer/combat aircraft,6.0,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0,Aircraft,Fighter/Combat Aircraft
1,Afghanistan,Russia,2004.0,6.0,aircraft engine,aircraft engine,6.0,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0,Specialized Equipment,Aircraft Engines
2,Afghanistan,Soviet Union,1977.0,26.0,AI-25,turbofan,26.0,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6,Specialized Equipment,Aircraft Engines
3,Afghanistan,Soviet Union,1988.0,2300.0,R-17 Elbrus,SSM,2300.0,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0,Missiles/Rockets/Bombs,Surface-to-Surface Missiles (SSM)
4,Afghanistan,United Kingdom,2009.0,2.0,Mi-17,transport helicopter,2.0,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8,Helicopters,Transport/Utility Helicopters


In [224]:
df = df.drop(columns=['.1', '.2'], errors='ignore')

df['Year of order'] = df['Year of order'].astype(int)

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29058 entries, 0 to 29057
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Recipient                       29056 non-null  object 
 1   Supplier                        29058 non-null  object 
 2   Year of order                   29058 non-null  int64  
 3   Number ordered                  28827 non-null  float64
 4   Weapon designation              29056 non-null  object 
 5   Weapon description              29056 non-null  object 
 6   Number delivered                29056 non-null  float64
 7   Year(s) of delivery             27951 non-null  object 
 8   status                          29056 non-null  object 
 9   Comments                        23831 non-null  object 
 10  SIPRI TIV per unit              29054 non-null  float64
 11  SIPRI TIV for total order       29054 non-null  float64
 12  SIPRI TIV of delivered weapons  

In [226]:
df.to_csv('final_data/cleaned_transfer_data.csv')

## Restructure dataframe
We need to have a primary country name so we restructure the dataframe so that there are two rows per transaction one where the supplier is the primary country and one where the recipient is the primary country. 

In [227]:
# Melt the DataFrame to create rows for Supplier and Recipient
df["transaction_id"] = range(1, len(df) + 1)

supplier_df = df.copy()
supplier_df["Country"] = supplier_df["Supplier"]
supplier_df["Country Role"] = "Supplier"
#supplier_df.drop(columns=["Supplier", "Recipient"], inplace=True)

recipient_df = df.copy()
recipient_df["Country"] = recipient_df["Recipient"]
recipient_df["Country Role"] = "Recipient"
#recipient_df.drop(columns=["Supplier", "Recipient"], inplace=True)





In [228]:
# Combine the two DataFrames
transformed_df = pd.concat([supplier_df, recipient_df], ignore_index=True)

# Reorder columns to have "Country" and "Country Role" as the first two columns
transformed_df["SIPRI TIV for total order - Pos_Neg"] = np.where(
    transformed_df["Country Role"] == "Recipient",
    transformed_df["SIPRI TIV for total order"] * -1.0,
    transformed_df["SIPRI TIV for total order"]
)

drop_cols = ['Number delivered','Year(s) of delivery','SIPRI TIV of delivered weapons']
transformed_df = transformed_df.drop(columns=drop_cols)
transformed_df["Partner Country"] = np.where(
    transformed_df["Recipient"] != transformed_df["Country"],  # Condition
    transformed_df["Recipient"],                  # Value if True
    transformed_df["Supplier"]                   # Value if False
)
transformed_df = transformed_df.drop(columns=["Supplier", "Recipient"])
cols = ["transaction_id", "Country", "Country Role", "Partner Country"] + [col for col in transformed_df.columns if col not in ["transaction_id", "Country", "Country Role", "Partner Country"]]

transformed_df = transformed_df[cols]

transformed_df.sort_values('transaction_id').head()


,transaction_id,Country,Country Role,Partner Country,Year of order,Number ordered,Weapon designation,Weapon description,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,Category,Sub-Category,SIPRI TIV for total order - Pos_Neg
0,1,Brazil,Supplier,Afghanistan,2017,6.0,EMB-314 Super Tucano,trainer/combat aircraft,New,A-29B version; financed by USA; ordered via US...,4.5,27.0,Aircraft,Fighter/Combat Aircraft,27.0
29058,1,Afghanistan,Recipient,Brazil,2017,6.0,EMB-314 Super Tucano,trainer/combat aircraft,New,A-29B version; financed by USA; ordered via US...,4.5,27.0,Aircraft,Fighter/Combat Aircraft,-27.0
1,2,Russia,Supplier,Afghanistan,2004,6.0,aircraft engine,aircraft engine,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.5,3.0,Specialized Equipment,Aircraft Engines,3.0
29059,2,Afghanistan,Recipient,Russia,2004,6.0,aircraft engine,aircraft engine,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.5,3.0,Specialized Equipment,Aircraft Engines,-3.0
2,3,Soviet Union,Supplier,Afghanistan,1977,26.0,AI-25,turbofan,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.6,15.6,Specialized Equipment,Aircraft Engines,15.6


In [229]:
sipri_wdi_merge = df.merge(non_region_df, how='left', left_on=['Recipient','Year of order'], right_on=['sipri_country_mapping','Year'])

In [230]:
non_region_df['Year'] = non_region_df['Year'].astype(int)

non_region_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5112 entries, 40 to 17288
Data columns (total 13 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   Country Name                                                5112 non-null   object
 1   Country Code                                                5112 non-null   object
 2   Year                                                        5112 non-null   int64 
 3   Region                                                      5112 non-null   object
 4   Income Group                                                5088 non-null   object
 5   Armed forces personnel (% of total labor force)             3400 non-null   object
 6   Armed forces personnel, total                               3440 non-null   object
 7   Arms exports (SIPRI trend indicator values)                 981 non-null    object
 8   Arms import

In [231]:
melt_wdi_sipri_merge = transformed_df.merge(non_region_df, how='left', left_on=['Country','Year of order'], right_on=['sipri_country_mapping','Year'])
melt_wdi_sipri_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58116 entries, 0 to 58115
Data columns (total 28 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   transaction_id                                              58116 non-null  int64  
 1   Country                                                     58114 non-null  object 
 2   Country Role                                                58116 non-null  object 
 3   Partner Country                                             58112 non-null  object 
 4   Year of order                                               58116 non-null  int64  
 5   Number ordered                                              57654 non-null  float64
 6   Weapon designation                                          58112 non-null  object 
 7   Weapon description                                          58112 non-null  object 
 

In [232]:
drop_cols = ['Year', 'Region', 'Income Group', 'Country Name', 'sipri_country_mapping']
melt_wdi_sipri_merge = melt_wdi_sipri_merge.drop(columns=drop_cols)
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(country_region_mapping, how='left', left_on='Country', right_on='sipri_country_mapping')
melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'Region':'Primary Country Region',
                                    'Income Group':'Primary Country Income Group',
                                    'two_alpha_code':'primary_two_alpha_code',
                                    #'sipri_country_mapping_x':'sipri_country_mapping'
                                   }, axis=1, inplace=True)
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(country_region_mapping, how='left', left_on='Partner Country', right_on='sipri_country_mapping')
melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'Region':'Partner Country Region',
                                    'Income Group':'Partner Country Income Group',
                                    'two_alpha_code':'partner_two_alpha_code',
                                   # 'sipri_country_mapping_x':'sipri_country_mapping'
                                   }, axis=1, inplace=True)

melt_wdi_sipri_merge

,transaction_id,Country,Country Role,Partner Country,Year of order,Number ordered,Weapon designation,Weapon description,status,Comments,...,Arms imports (SIPRI trend indicator values),Military expenditure (% of GDP),Military expenditure (% of general government expenditure),Military expenditure (current LCU),Primary Country Region,Primary Country Income Group,primary_two_alpha_code,Partner Country Region,Partner Country Income Group,partner_two_alpha_code
0,1,Brazil,Supplier,Afghanistan,2017,6.0,EMB-314 Super Tucano,trainer/combat aircraft,New,A-29B version; financed by USA; ordered via US...,...,94000000.0,1.414022,3.69249,93120100000.0,Latin America & Caribbean,Upper middle income,BR,South Asia,Low income,AF
1,2,Russia,Supplier,Afghanistan,2004,6.0,aircraft engine,aircraft engine,New,Klimov TV-3-117 turboshaft; spare engines for ...,...,NaN,3.300354,11.170381,603803000000.0,Europe & Central Asia,High income,RU,South Asia,Low income,AF
2,3,Soviet Union,Supplier,Afghanistan,1977,26.0,AI-25,turbofan,New,For 26 L-39 trainer aircraft from Czechoslovak...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Asia,Low income,AF
3,4,Soviet Union,Supplier,Afghanistan,1988,2300.0,R-17 Elbrus,SSM,New,Mainly for use against Mujahideen rebel forces,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Asia,Low income,AF
4,5,United Kingdom,Supplier,Afghanistan,2009,2.0,Mi-17,transport helicopter,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,...,399000000.0,2.647725,5.949568,41089560430.972,Europe & Central Asia,High income,GB,South Asia,Low income,AF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58111,29054,Zimbabwe,Recipient,Russia,1998,6.0,Mi-24P/Mi-35P,combat helicopter,Second hand,Second-hand; bought for use in DRC against reb...,...,NaN,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income,ZW,Europe & Central Asia,High income,RU
58112,29055,Zimbabwe,Recipient,Italy,1967,20.0,Model-56 105mm,towed gun,New,Supplier uncertain,...,NaN,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income,ZW,Europe & Central Asia,High income,IT
58113,29056,Zimbabwe,Recipient,Soviet Union,1975,15.0,T-34-85,tank,Second hand,Second-hand; supplier uncertain,...,NaN,NaN,NaN,NaN,Sub-Saharan Africa,Lower middle income,ZW,NaN,NaN,NaN
58114,29057,Zimbabwe,Recipient,China,2004,5.0,Type-89/ZSD-89,APC,New,ARV version,...,14000000.0,2.705758,NaN,256100000.0,Sub-Saharan Africa,Lower middle income,ZW,East Asia & Pacific,Upper middle income,CN


In [233]:
melt_wdi_sipri_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58116 entries, 0 to 58115
Data columns (total 29 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   transaction_id                                              58116 non-null  int64  
 1   Country                                                     58114 non-null  object 
 2   Country Role                                                58116 non-null  object 
 3   Partner Country                                             58112 non-null  object 
 4   Year of order                                               58116 non-null  int64  
 5   Number ordered                                              57654 non-null  float64
 6   Weapon designation                                          58112 non-null  object 
 7   Weapon description                                          58112 non-null  object 
 

## Add Latitude and Longitude for each country
We need latitude and longitude to create lines between countries in tableau

In [234]:
lat_long_df = pd.read_csv('raw_data/countries_lat_long.csv')
lat_long_df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [235]:
lat_long_df[lat_long_df['country']=='TR']

,country,latitude,longitude,name
219,TR,38.963745,35.243322,Turkey


In [236]:
lat_long_df = lat_long_df[~lat_long_df['country'].isna()]

In [237]:
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(lat_long_df, how='left', left_on='primary_two_alpha_code', right_on='country')
melt_wdi_sipri_merge.drop(columns=['country','name'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'latitude':'Primary Latitude',
                             'longitude':'Primary Longitude'},
                             axis=1, inplace=True)

melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(lat_long_df, how='left', left_on='partner_two_alpha_code', right_on='country')
melt_wdi_sipri_merge.drop(columns=['country','name'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'latitude':'Partner Latitude',
                             'longitude':'Partner Longitude'},
                             axis=1, inplace=True)
melt_wdi_sipri_merge.head()

,transaction_id,Country,Country Role,Partner Country,Year of order,Number ordered,Weapon designation,Weapon description,status,Comments,...,Primary Country Region,Primary Country Income Group,primary_two_alpha_code,Partner Country Region,Partner Country Income Group,partner_two_alpha_code,Primary Latitude,Primary Longitude,Partner Latitude,Partner Longitude
0,1,Brazil,Supplier,Afghanistan,2017,6.0,EMB-314 Super Tucano,trainer/combat aircraft,New,A-29B version; financed by USA; ordered via US...,...,Latin America & Caribbean,Upper middle income,BR,South Asia,Low income,AF,-14.235004,-51.925280,33.93911,67.709953
1,2,Russia,Supplier,Afghanistan,2004,6.0,aircraft engine,aircraft engine,New,Klimov TV-3-117 turboshaft; spare engines for ...,...,Europe & Central Asia,High income,RU,South Asia,Low income,AF,61.524010,105.318756,33.93911,67.709953
2,3,Soviet Union,Supplier,Afghanistan,1977,26.0,AI-25,turbofan,New,For 26 L-39 trainer aircraft from Czechoslovak...,...,NaN,NaN,NaN,South Asia,Low income,AF,NaN,NaN,33.93911,67.709953
3,4,Soviet Union,Supplier,Afghanistan,1988,2300.0,R-17 Elbrus,SSM,New,Mainly for use against Mujahideen rebel forces,...,NaN,NaN,NaN,South Asia,Low income,AF,NaN,NaN,33.93911,67.709953
4,5,United Kingdom,Supplier,Afghanistan,2009,2.0,Mi-17,transport helicopter,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,...,Europe & Central Asia,High income,GB,South Asia,Low income,AF,55.378051,-3.435973,33.93911,67.709953


In [238]:
melt_wdi_sipri_merge = melt_wdi_sipri_merge[melt_wdi_sipri_merge['Year of order']>=1960]
melt_wdi_sipri_merge.head()

,transaction_id,Country,Country Role,Partner Country,Year of order,Number ordered,Weapon designation,Weapon description,status,Comments,...,Primary Country Region,Primary Country Income Group,primary_two_alpha_code,Partner Country Region,Partner Country Income Group,partner_two_alpha_code,Primary Latitude,Primary Longitude,Partner Latitude,Partner Longitude
0,1,Brazil,Supplier,Afghanistan,2017,6.0,EMB-314 Super Tucano,trainer/combat aircraft,New,A-29B version; financed by USA; ordered via US...,...,Latin America & Caribbean,Upper middle income,BR,South Asia,Low income,AF,-14.235004,-51.925280,33.93911,67.709953
1,2,Russia,Supplier,Afghanistan,2004,6.0,aircraft engine,aircraft engine,New,Klimov TV-3-117 turboshaft; spare engines for ...,...,Europe & Central Asia,High income,RU,South Asia,Low income,AF,61.524010,105.318756,33.93911,67.709953
2,3,Soviet Union,Supplier,Afghanistan,1977,26.0,AI-25,turbofan,New,For 26 L-39 trainer aircraft from Czechoslovak...,...,NaN,NaN,NaN,South Asia,Low income,AF,NaN,NaN,33.93911,67.709953
3,4,Soviet Union,Supplier,Afghanistan,1988,2300.0,R-17 Elbrus,SSM,New,Mainly for use against Mujahideen rebel forces,...,NaN,NaN,NaN,South Asia,Low income,AF,NaN,NaN,33.93911,67.709953
4,5,United Kingdom,Supplier,Afghanistan,2009,2.0,Mi-17,transport helicopter,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,...,Europe & Central Asia,High income,GB,South Asia,Low income,AF,55.378051,-3.435973,33.93911,67.709953


In [239]:
melt_wdi_sipri_merge[melt_wdi_sipri_merge['Primary Latitude'].isna()]['Country'].unique()

array(['Soviet Union', 'Czechoslovakia', 'unknown supplier(s)',
       'East Germany (GDR)', 'Yugoslavia', "Cote d'Ivoire", 'Libya HoR*',
       'United Nations**', 'South Yemen', 'South Vietnam',
       'European Union**', 'FMLN (El Salvador)*', 'Cyprus', 'North Yemen',
       'Mujahedin (Afghanistan)*', 'African Union**', 'Amal (Lebanon)*',
       'ANC (South Africa)*', 'Anti-Castro rebels (Cuba)*', 'Biafra',
       'Contras (Nicaragua)*', 'Darfur rebels (Sudan)*',
       'ELF (Ethiopia)*', 'EPLF (Ethiopia)*', 'FAN (Chad)*',
       'FNLA (Angola)*', 'FRELIMO (Portugal)*', 'GUNT (Chad)*',
       'Haiti rebels*', 'Hamas (Palestine)*', 'Hezbollah (Lebanon)*',
       'Houthi rebels (Yemen)*', 'Katanga', 'Khmer Rouge (Cambodia)*',
       'Kurdistan Regional Government (Iraq)*',
       'Lebanon Palestinian rebels*', 'LF (Lebanon)*', 'Libya GNC',
       'LRA (Uganda)*', 'LTTE (Sri Lanka)*', 'MNLF (Philippines)*',
       'MPLA (Portugal)*', 'MTA (Myanmar)*', 'Namibia', 'NATO**',
       'NLA 

In [240]:
#melt_wdi_sipri_merge = melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'])

melt_wdi_sipri_merge.to_csv('final_data/'+current_date+'one_row_one_country_data.csv')

In [242]:
melt_wdi_sipri_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52956 entries, 0 to 58115
Data columns (total 33 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   transaction_id                                              52956 non-null  int64  
 1   Country                                                     52956 non-null  object 
 2   Country Role                                                52956 non-null  object 
 3   Partner Country                                             52956 non-null  object 
 4   Year of order                                               52956 non-null  int64  
 5   Number ordered                                              52498 non-null  float64
 6   Weapon designation                                          52956 non-null  object 
 7   Weapon description                                          52956 non-null  object 
 8   s

In [243]:
#sipri_wdi_merge.to_csv('final_data/final_nov21_sipri_recipient_indicator_merge.csv')

In [244]:
#sipri_wdi_merge[(sipri_wdi_merge['sipri_country_mapping'].isna())&(sipri_wdi_merge['Year of order']>2000)]

In [245]:
#sipri_wdi_merge[(sipri_wdi_merge['sipri_country_mapping'].isna())&(sipri_wdi_merge['Year of order']>2000)]['Recipient'].value_counts()

In [246]:
#non_region_df[(non_region_df['Year']==2007)&(non_region_df['sipri_country_mapping']=='Turkey')]

## Create csv with one row per year delivered

In [247]:
# delivered_df = df.dropna(subset=['First Year'])

# delivered_df['First Year'] = delivered_df['First Year'].astype(int)

# # Expand rows based on `Num years Delivered In`
# annual_delivery_df = delivered_df.loc[delivered_df.index.repeat(delivered_df['Num years Delivered In'])].copy()

# # Calculate the `year_delivered` for each expanded row
# annual_delivery_df['year_delivered'] = annual_delivery_df.groupby(level=0).cumcount() + annual_delivery_df['First Year']
# annual_delivery_df['num_delivered_in_year'] = annual_delivery_df['Number delivered']/annual_delivery_df['Num years Delivered In']
# annual_delivery_df['SIPRI_TIV_delivered_in_year'] = annual_delivery_df['num_delivered_in_year']*annual_delivery_df['SIPRI TIV per unit']


In [248]:
# annual_delivery_df = annual_delivery_df.rename(columns={'Number delivered':'Total number delivered'})
# annual_delivery_df.head()

In [249]:
# annual_delivery_df.to_csv('final_data/annual_delivery_data.csv')